In [13]:
import keplergl
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle
seed = 4

# Distribution

In [14]:
with open('./model/origen.pkl', 'rb') as f:
    originModel = pickle.load(f)

with open('./model/destino.pkl', 'rb') as f:
    destinationModel = pickle.load(f)

toPredictData = pd.read_csv('./data/selectedDataV2.csv', index_col='CODIGO_MZ')
toPredictData.drop([x for x in toPredictData if 'datosAgrupados' in x], axis = 1, inplace=True)
toJoin = toPredictData.copy()
toJoin['Viajes Origen'] = originModel.predict(toPredictData)
toJoin['Viajes Destino'] = destinationModel.predict(toPredictData)
toJoin['Viajes Origen'] = np.int64(round(toJoin['Viajes Origen'], 0))
toJoin['Viajes Destino'] = np.int64(round(toJoin['Viajes Destino'], 0))
toJoin.drop(toPredictData.columns, axis = 1, inplace = True)
toJoin

,Viajes Origen,Viajes Destino
CODIGO_MZ,,
353,5136,5109
352,804,833
205,11293,11286
486,4145,4118
489,773,777
...,...,...
158,21419,21649
165,16937,16912
151,15916,15790


# Flux

In [2]:
data = pd.read_csv('./data/fluxModelData.csv', index_col=['Origen', 'Destino'])
data

Caminando  Transporte Colectivo  Taxi  Bicicleta  Motocicleta  \
Origen Destino                                                                  
1.0    1.0         2175.0                 623.0   0.0        0.0          0.0   
       2.0          212.0                 147.0   0.0        0.0          0.0   
       3.0            0.0                 165.0   0.0        0.0          0.0   
       4.0            0.0                 311.0   0.0        0.0          0.0   
       14.0           0.0                 311.0   0.0        0.0          0.0   
...                   ...                   ...   ...        ...          ...   
490.0  403.0          0.0                   0.0   0.0        0.0          0.0   
       406.0          0.0                   0.0   0.0        0.0          0.0   
       446.0          0.0                   0.0   0.0        0.0          0.0   
       458.0          0.0                   0.0   0.0        0.0          0.0   
       490.0        200.0                   0.0   0.0        0.0          0.0   

                Vehiculo  Otros  Total  \
Origen Destino                           
1.0    1.0           0.0    0.0   2798   
       2.0         185.0    0.0    544   
       3.0           0.0    0.0    165   
       4.0           0.0    0.0    311   
       14.0          0.0    0.0    311   
...                  ...    ...    ...   
490.0  403.0       200.0    0.0    200   
       406.0       453.0    0.0    453   
       446.0       520.0    0.0    520   
       458.0       142.0    0.0    142   
       490.0       400.0    0.0    600   

                datosAgrupados_Caminando_destino__ORIGEN  \
Origen Destino                                             
1.0    1.0                                        4663.0   
       2.0                                        4663.0   
       3.0                                        4663.0   
       4.0                                        4663.0   
       14.0                                       4663.0   
...                                                  ...   
490.0  403.0                                       200.0   
       406.0                                       200.0   
       446.0                                       200.0   
       458.0                                       200.0   
       490.0                                       200.0   

                datosAgrupados_Transporte_Colectivo_destino__ORIGEN  ...  \
Origen Destino                                                       ...   
1.0    1.0                                                 5428.0    ...   
       2.0                                                 5428.0    ...   
       3.0                                                 5428.0    ...   
       4.0                                                 5428.0    ...   
       14.0                                                5428.0    ...   
...                                                           ...    ...   
490.0  403.0                                               1051.0    ...   
       406.0                                               1051.0    ...   
       446.0                                               1051.0    ...   
       458.0                                               1051.0    ...   
       490.0                                               1051.0    ...   

                sum_VPH_CVJ__DESTINO  sum_VPH_SINRTV__DESTINO  \
Origen Destino                                                  
1.0    1.0                     234.0                     10.0   
       2.0                     128.0                      6.0   
       3.0                     167.0                      0.0   
       4.0                      28.0                      3.0   
       14.0                    279.0                      6.0   
...                              ...                      ...   
490.0  403.0                  2093.0                     21.0   
       406.0                   486.0                     34.

In [3]:
targets = ['Caminando', 'Transporte Colectivo', 'Taxi', 'Bicicleta', 'Motocicleta', 'Vehiculo', 'Otros', 'Total']
categorical = data.select_dtypes(include=['object']).columns
codes = [x for x in data.columns if 'CODIGO' in x]
predictors = [x for x in data.columns if x not in targets and x not in categorical and x not in codes]

In [4]:
with open('./model/flux.pkl', 'rb') as f:
    fluxModel = pickle.load(f)

In [5]:
x = pd.DataFrame(fluxModel.predict(data[predictors]), index=data[predictors].index, columns=data[targets].columns)
x[x < 0] = 0
x = x.astype(int)
x

555/555 [==============================] - 1s 2ms/step


Caminando  Transporte Colectivo  Taxi  Bicicleta  Motocicleta  \
Origen Destino                                                                  
1.0    1.0           1955                   170     8         33           22   
       2.0             91                    76    69          0           67   
       3.0             74                   155     4          0            0   
       4.0              6                    94    91          0           81   
       14.0            71                   104    19          0            2   
...                   ...                   ...   ...        ...          ...   
490.0  403.0          118                     0     9          0            0   
       406.0           76                    23    21          0            0   
       446.0           69                     0    24          0            0   
       458.0            0                    44    39         27            0   
       490.0         1042                    95     1        181           12   

                Vehiculo  Otros  Total  
Origen Destino                          
1.0    1.0           114      0   2337  
       2.0             0      0    413  
       3.0            55      0    260  
       4.0             0      0    325  
       14.0           85      0    191  
...                  ...    ...    ...  
490.0  403.0         173      0    255  
       406.0         170      0    279  
       446.0         222      0    310  
       458.0         135     13    217  
       490.0         646      0   1936  

[17737 rows x 8 columns]

# Map

## Viajes OD

In [47]:
zonas = gpd.read_file('./data/Zonificacion/Zonificacion.shp')
zonas

,ID,AREA,ID_NODO,CODIGO_MZ,MPIO,POB2005,POB2010,POB2000,POB2012,VIAJES_AUT,VIAJES_TP,AreaHA,geometry
0,353,0.66,23070,353,Tonal,950.0,1097.0,547.0,1262.91,1270.0,1311.0,65.679518,"POLYGON ((-103.23486 20.66894, -103.23618 20.6..."
1,352,34.44,23043,352,Tonal,0.0,12275.0,698.0,13150.56,NaN,NaN,3405.625225,"MULTIPOLYGON (((-103.22730 20.67261, -103.2273..."
2,178,8.37,33263,178,Guada,46.0,46.0,0.0,46.00,NaN,NaN,827.453665,"POLYGON ((-103.26569 20.70559, -103.26580 20.7..."
3,205,136.13,1057,205,Juana,3696.0,3921.0,0.0,4015.00,0.0,169.0,13465.127422,"POLYGON ((-103.09714 20.53096, -103.09697 20.5..."
4,486,15.55,23383,486,Ext,0.0,0.0,0.0,0.00,2182.0,1204.0,1538.143448,"POLYGON ((-103.09714 20.53096, -103.09777 20.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,158,0.51,27449,158,Guada,10146.0,9805.0,11184.0,9550.70,2644.0,5504.0,50.319803,"POLYGON ((-103.37577 20.62992, -103.37549 20.6..."
474,165,0.67,12476,165,Guada,11694.0,10758.0,11516.0,10613.53,5127.0,7060.0,66.570406,"POLYGON ((-103.35993 20.62026, -103.35983 20.6..."
475,151,0.53,12445,151,Guada,6973.0,6734.0,7203.0,6643.93,3089.0,4687.0,52.177476,"POLYGON ((-103.34855 20.62832, -103.34726 20.6..."
476,492,74.37,20428,349,Tonal,16848.0,11736.0,1267.0,12573.11,3188.0,13649.0,7354.580057,"POLYGON ((-103.25581 20.55973, -103.25603 20.5..."


In [11]:
toDrop = [x for x in zonas.columns if x not in ['CODIGO_MZ', 'geometry']]
zonasMod = zonas.drop(toDrop, axis = 1)
zonasMod

,CODIGO_MZ,geometry
0,353,"POLYGON ((-103.23486 20.66894, -103.23618 20.6..."
1,352,"MULTIPOLYGON (((-103.22730 20.67261, -103.2273..."
2,178,"POLYGON ((-103.26569 20.70559, -103.26580 20.7..."
3,205,"POLYGON ((-103.09714 20.53096, -103.09697 20.5..."
4,486,"POLYGON ((-103.09714 20.53096, -103.09777 20.5..."
...,...,...
473,158,"POLYGON ((-103.37577 20.62992, -103.37549 20.6..."
474,165,"POLYGON ((-103.35993 20.62026, -103.35983 20.6..."
475,151,"POLYGON ((-103.34855 20.62832, -103.34726 20.6..."
476,349,"POLYGON ((-103.25581 20.55973, -103.25603 20.5..."


In [17]:
zonasMod = zonasMod.join(toJoin, on='CODIGO_MZ')
zonasMod

,CODIGO_MZ,geometry,Viajes Origen,Viajes Destino
0,353,"POLYGON ((-103.23486 20.66894, -103.23618 20.6...",5136,5109
1,352,"MULTIPOLYGON (((-103.22730 20.67261, -103.2273...",804,833
2,178,"POLYGON ((-103.26569 20.70559, -103.26580 20.7...",21,-6
3,205,"POLYGON ((-103.09714 20.53096, -103.09697 20.5...",11293,11286
4,486,"POLYGON ((-103.09714 20.53096, -103.09777 20.5...",4145,4118
...,...,...,...,...
473,158,"POLYGON ((-103.37577 20.62992, -103.37549 20.6...",21419,21649
474,165,"POLYGON ((-103.35993 20.62026, -103.35983 20.6...",16937,16912
475,151,"POLYGON ((-103.34855 20.62832, -103.34726 20.6...",15916,15790
476,349,"POLYGON ((-103.25581 20.55973, -103.25603 20.5...",35643,35664


In [18]:
map_1 = keplergl.KeplerGl()
map_1.add_data(data=zonasMod, name="Zonas")
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Zonas': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

## Distribución de Viajes

In [22]:
xy = pd.read_csv('./data/Zonificacion/zonificacionXY.csv', index_col='CODIGO_MZ')
xy.drop('OID_', axis=1, inplace=True)
xy

,PX,PY
CODIGO_MZ,,
353,-103.232185,20.672608
352,-103.204295,20.652854
178,-103.298114,20.737875
205,-103.134625,20.481134
486,-103.101954,20.547243
...,...,...
158,-103.372280,20.631748
165,-103.353927,20.622016
151,-103.352320,20.627166


In [37]:
fluxData = x.join(xy, 'Origen').join(xy, 'Destino', lsuffix='_Origen', rsuffix='_Destino')
fluxData

Caminando  Transporte Colectivo  Taxi  Bicicleta  Motocicleta  \
Origen Destino                                                                  
1.0    1.0           1955                   170     8         33           22   
       2.0             91                    76    69          0           67   
       3.0             74                   155     4          0            0   
       4.0              6                    94    91          0           81   
       14.0            71                   104    19          0            2   
...                   ...                   ...   ...        ...          ...   
490.0  403.0          118                     0     9          0            0   
       406.0           76                    23    21          0            0   
       446.0           69                     0    24          0            0   
       458.0            0                    44    39         27            0   
       490.0         1042                    95     1        181           12   

                Vehiculo  Otros  Total   PX_Origen  PY_Origen  PX_Destino  \
Origen Destino                                                              
1.0    1.0           114      0   2337 -103.356189  20.682380 -103.356189   
       2.0             0      0    413 -103.356189  20.682380 -103.344735   
       3.0            55      0    260 -103.356189  20.682380 -103.354770   
       4.0             0      0    325 -103.356189  20.682380 -103.346562   
       14.0           85      0    191 -103.356189  20.682380 -103.367095   
...                  ...    ...    ...         ...        ...         ...   
490.0  403.0         173      0    255 -103.667941  20.744986 -103.434351   
       406.0         170      0    279 -103.667941  20.744986 -103.426355   
       446.0         222      0    310 -103.667941  20.744986 -103.426656   
       458.0         135     13    217 -103.667941  20.744986 -103.577366   
       490.0         646      0   1936 -103.667941  20.744986 -103.667941   

                PY_Destino  
Origen Destino              
1.0    1.0       20.682380  
       2.0       20.678920  
       3.0       20.672806  
       4.0       20.673044  
       14.0      20.677844  
...                    ...  
490.0  403.0     20.744215  
       406.0     20.725370  
       446.0     20.603981  
       458.0     20.706824  
       490.0     20.744986  

[17737 rows x 12 columns]

In [38]:
map_2 = keplergl.KeplerGl()
map_2.add_data(data=zonas.drop(toDrop, axis = 1), name="Zonas")
map_2.add_data(data=fluxData, name = 'FlujoViajes')
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Zonas': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

In [39]:
map_2.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '2um4',
     'type': 'geojson',
     'config': {'dataId': 'Zonas',
      'label': 'Zonas',
      'color': [248, 149, 112],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [130, 154, 227],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0,

## Clean Maps

In [33]:
config1 = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'farkchy',
     'type': 'geojson',
     'config': {'dataId': 'Zonas',
      'label': 'Zonas_Destino',
      'color': [130, 154, 227],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.54,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Uber Viz Diverging 1.5',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#00939C',
         '#5DBABF',
         '#BAE1E2',
         '#F8C0AA',
         '#DD7755',
         '#C22E00']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': True,
       'filled': True,
       'enable3d': True,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'Viajes Destino',
       'type': 'integer'},
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': {'name': 'Viajes Destino', 'type': 'integer'},
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}},
    {'id': 'rlxa3os',
     'type': 'geojson',
     'config': {'dataId': 'Zonas',
      'label': 'Zonas_Origen',
      'color': [130, 154, 227],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.54,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Uber Viz Diverging 1.5',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#00939C',
         '#5DBABF',
         '#BAE1E2',
         '#F8C0AA',
         '#DD7755',
         '#C22E00']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': True,
       'filled': True,
       'enable3d': True,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'Viajes Origen',
       'type': 'integer'},
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': {'name': 'Viajes Origen', 'type': 'integer'},
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'Zonas': [{'name': 'CODIGO_MZ',
        'format': None},
       {'name': 'Viajes Origen', 'format': None},
       {'name': 'Viajes Destino', 'format': None}],
      'Copy of Zonas': [{'name': 'CODIGO_MZ', 'format': None},
       {'name': 'Viajes Origen', 'format': None},
       {'name': 'Viajes Destino', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [{'layers': {'farkchy': False, 'rlxa3os': True}},
    {'layers': {'farkchy': True, 'rlxa3os': False}}],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 24,
   'dragRotate': True,
   'latitude': 20.55202873488371,
   'longitude': -103.36770186662218,
   'pitch': 50,
   'zoom': 9,
   'isSplit': True},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [41]:
map_clean1 = keplergl.KeplerGl(data={'Zonas': zonasMod, 'Copy of Zonas': zonasMod}, config = config1)
map_clean1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'farkchy', 'type': …

In [49]:
map_clean1.save_to_html(file_name='C:\\Users\\Rafael\\OneDrive - ITESO\\2023.3 Otoño\\PAP\\data\\tmp\\tool\\Reports\\OD_Data.html')

Map saved to C:\Users\Rafael\OneDrive - ITESO\2023.3 Otoño\PAP\data\tmp\tool\Reports\OD_Data.html!


In [40]:
config2 = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '2um4',
     'type': 'geojson',
     'config': {'dataId': 'Zonas',
      'label': 'Zonas',
      'color': [248, 149, 112],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [130, 154, 227],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': True,
       'filled': False,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}},
    {'id': 'od9z5r',
     'type': 'line',
     'config': {'dataId': 'FlujoViajes',
      'label': 'Flujo',
      'color': [231, 159, 213],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat0': 'PY_Origen',
       'lng0': 'PX_Origen',
       'lat1': 'PY_Destino',
       'lng1': 'PX_Destino',
       'alt0': None,
       'alt1': None},
      'isVisible': True,
      'visConfig': {'opacity': 0.15,
       'thickness': 2,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0, 13.4],
       'targetColor': [82, 163, 83],
       'elevationScale': 0},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': {'name': 'Total', 'type': 'integer'},
      'sizeScale': 'sqrt'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'Zonas': [{'name': 'CODIGO_MZ',
        'format': None}],
      'FlujoViajes': [{'name': 'Caminando', 'format': None},
       {'name': 'Transporte Colectivo', 'format': None},
       {'name': 'Taxi', 'format': None},
       {'name': 'Bicicleta', 'format': None},
       {'name': 'Motocicleta', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 20.569629840077784,
   'longitude': -103.68190457934014,
   'pitch': 0,
   'zoom': 8.349332977080866,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [44]:
map_clean2 = keplergl.KeplerGl(data={'Zonas': zonas.drop(toDrop, axis = 1), 'FlujoViajes': fluxData}, config = config2)
map_clean2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '2um4', 'type': 'ge…